In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import ttest_ind
from scipy import stats
from scipy.stats import mannwhitneyu

In [3]:
# Replace 'otu_file' with the path to your TSV file
otu_file = 'otu_table_psn_v35.txt'
# wget http://downloads.hmpdacc.org/data/HMQCP/otu_table_psn_v35.txt.gz
# Read the TSV file while skipping the first line
otu_data = pd.read_csv(otu_file, sep='\t', skiprows=1)
# otu_data = pd.read_csv("path_to_your_output_file.csv")
otu_data.columns.values[0] = 'otu_id'

# Define a function to parse the taxonomic data
def parse_tax_data(otu_data, class_cols, class_regex, class_key, class_sep):
    # Extract class information based on class_cols and class_sep
    tax_data = otu_data[class_cols].str.split(class_sep, expand=True)
    
    # Create a DataFrame for taxonomic classification
    tax_data.columns = [f'{class_key["hmp_rank"]}_{i}' for i in range(tax_data.shape[1])]
    tax_data[class_key["hmp_tax"]] = tax_data.apply(lambda row: class_sep.join(row.dropna()), axis=1)
    
    # Merge taxonomic data back with the original OTU data
    otu_data = otu_data.drop(columns=[class_cols])
    otu_data = pd.concat([otu_data, tax_data], axis=1)
    
    return otu_data

# # Apply the function to parse taxonomic data
hmp_data = parse_tax_data(otu_data, "Consensus Lineage", r"([a-z]{0,1})_{0,2}(.*)$", {"hmp_rank": "taxon_rank", "hmp_tax": "taxon_name"}, ";")

# # Remove the regex match table (not needed in this case)
otu_data = hmp_data.drop(columns=[col for col in hmp_data.columns if col.startswith('taxon_rank_')])

# # Rename the abundance matrix to something more understandable
otu_data.columns = ['otu_count' if col == 'otu_id' else col for col in otu_data.columns]
# Select the top 1000 rows
# top_1000_hmp_data = hmp_data.head(5)


# Extract the suffix of 'g__' in the 'taxon_name' column
otu_data['genus'] = otu_data['taxon_name'].str.extract(r'g__([^;]*)')
otu_data = otu_data[otu_data['genus'].notna() & (otu_data['genus'] != '')]
# # Delete the 'taxon_name' column
otu_data = otu_data.drop(columns=['taxon_name'])
# # Group by 'genus' and accumulate the OTU counts
if 'genus' in otu_data.columns:
    otu_data = otu_data.groupby('genus').sum().reset_index()
else:
    otu_data = otu_data  # If there is no 'genus' column, use the original data

# # Print the filtered DataFrame
# print(otu_data.head(5))
otu_data = otu_data.drop(columns=['otu_count'])
num_rows, num_columns = otu_data.shape
# print(f'The DataFrame has {num_rows} rows and {num_columns} columns.')
# Rotate the DataFrame 90 degrees clockwise
otu_data = otu_data.transpose()

# Make the first row the header
otu_data.columns = otu_data.iloc[0]
otu_data = otu_data[1:]
otu_data.to_csv("genus_rotated.csv", index=False)
# Define the path for the output CSV file
# csv_file = 'path_to_your_output_file.csv'

# # Convert the DataFrame to a CSV file
# top_1000_hmp_data.to_csv(csv_file, index=False)


In [16]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# Load the OTU table file
otu_table = pd.read_csv('genus_rotated.csv', header=0, index_col=0)
# print(otu_table.head(2))
# Create a dictionary to map genus names to unique indices
genus_names = otu_table.columns.tolist()
genus_to_idx = {genus: idx for idx, genus in enumerate(genus_names)}

# print(genus_to_idx)
# Define a dataset class
class OTUDataset(Dataset):
    def __init__(self, data, genus_to_idx):
        self.data = data
        self.genus_to_idx = genus_to_idx
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        counts = self.data.iloc[idx].tolist()
        input_ids = [self.genus_to_idx[genus] for genus in self.genus_to_idx]
        
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'labels': torch.tensor(counts, dtype=torch.float)
        }

# Create a dataset and dataloader
dataset = OTUDataset(otu_table, genus_to_idx)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
# dataset

In [23]:
import time
# Define the transformer model
class SimpleTransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_encoder_layers, dim_feedforward, max_seq_length):
        super(SimpleTransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = nn.Embedding(max_seq_length, d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_encoder_layers)
        self.regressor = nn.Linear(d_model, vocab_size)
        
    def forward(self, src, labels=None):
        src = self.embedding(src) + self.pos_encoder(torch.arange(src.size(1), device=src.device).unsqueeze(0).repeat(src.size(0), 1))
        output = self.transformer_encoder(src)
        output = output[:, 0, :]  # Use the first token representation for regression
        logits = self.regressor(output)
        
        loss = None
        if labels is not None:
            loss = nn.MSELoss()(logits, labels)
        
        return loss, logits
# Define the model parameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vocab_size = len(genus_to_idx)
d_model = 256
nhead = 4
num_encoder_layers = 4
dim_feedforward = 512
max_seq_length = len(genus_to_idx)
model = SimpleTransformerModel(vocab_size, d_model, nhead, num_encoder_layers, dim_feedforward, max_seq_length).to(device)

# Training loop
optimizer = Adam(model.parameters(), lr=1e-4)
num_epochs = 300
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        loss, _ = model(input_ids, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

end_time = time.time()
training_time = end_time - start_time
print(f'Training Time: {training_time:.2f} seconds')

# Save the model
torch.save(model.state_dict(), 'simple_transformer_model.pth')

Epoch 1/300, Loss: 47983.7206
Epoch 2/300, Loss: 47668.8756
Epoch 3/300, Loss: 47257.5199
Epoch 4/300, Loss: 46860.5186
Epoch 5/300, Loss: 46481.2610
Epoch 6/300, Loss: 46063.7235
